In [1]:
import datajoint as dj

In [2]:
schema = dj.schema('university')

Connecting dimitri@localhost:3306


In [3]:
@schema
class Student(dj.Manual):
    definition = """
    student_id : int unsigned   # university-wide ID number
    ---
    first_name      : varchar(40)
    last_name       : varchar(40)
    sex             : enum('F', 'M', 'U')
    """

In [4]:
Student.insert1({
    'student_id': 1, 
    'first_name': 'Alice', 
    'last_name': 'Anderson', 
    'sex': 'F'})

In [5]:
Student.insert1((2, 'Bob', 'Dylan', 'M'))

In [6]:
Student()

student_id university-wide ID number,first_name,last_name,sex
1,Alice,Anderson,F
2,Bob,Dylan,M


In [7]:
Student.insert1((3, 'Carol', 'Lewis', 'F'))

In [8]:
Student.drop()

`university`.`student` (3 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [9]:
@schema
class Student(dj.Manual):
    definition = """
    student_id : int unsigned   # university-wide ID number
    ---
    first_name      : varchar(40)
    last_name       : varchar(40)
    sex             : enum('F', 'M', 'U')
    date_of_birth   : date
    home_address    : varchar(120) # mailing street address
    home_city       : varchar(60)  # mailing address
    home_state      : char(2)      # US state acronym: e.g. OH
    home_zip        : char(10)     # zipcode e.g. 93979-4979
    home_phone      : varchar(20)  # e.g. 414.657.6883x0881
    """

In [11]:
import faker
import random

def make_students():
    fake = faker.Faker()
    fake_name = {'F': fake.name_female, 'M': fake.name_male}
    while True:  # ignore invalid values
        try:
            sex = random.choice(('F', 'M'))
            first_name, last_name = fake_name[sex]().split(' ')[:2]
            street_address, city = fake.address().split('\n')
            city, state = city.split(', ')
            state, zipcode = state.split(' ')            
        except ValueError:
            continue
        else:
            yield dict(
                first_name=first_name,
                last_name=last_name,
                sex=sex,
                home_address=street_address,
                home_city=city,
                home_state=state,
                home_zip=zipcode,
                date_of_birth=str(
                    fake.date_time_between(start_date="-35y", end_date="-15y").date()),
                home_phone = fake.phone_number()[:20])
            
students = make_students()

In [12]:
next(students)

{'first_name': 'Sarah',
 'last_name': 'Abbott',
 'sex': 'F',
 'home_address': '464 Cabrera Valleys Suite 149',
 'home_city': 'Kennethhaven',
 'home_state': 'NC',
 'home_zip': '83300',
 'date_of_birth': '1995-12-19',
 'home_phone': '9208211564'}

In [13]:
import itertools
student_id = itertools.count()

In [14]:
for _, student in zip(range(300), students):
    Student.insert1(dict(student, student_id=next(student_id)))

In [15]:
Student.insert(
    dict(student, student_id=next(student_id)) for _, student in zip(range(300), students))

In [16]:
Student()

student_id university-wide ID number,first_name,last_name,sex,date_of_birth,home_address mailing street address,home_city mailing address,home_state US state acronym: e.g. OH,home_zip zipcode e.g. 93979-4979,home_phone e.g. 414.657.6883x0881
0,Eric,Anderson,M,2000-06-14,238 Clark Trafficway Apt. 820,North Jacob,ID,24273,+1-139-516-5686x245
1,Jamie,Gomez,F,1991-12-31,340 Manuel Lake Apt. 728,North William,MI,94138,522.492.8344
2,Mitchell,Robertson,M,1993-01-14,35319 Michael Forks,Leahborough,VT,94553,382-870-7338x311
3,Jason,Mosley,M,1994-08-28,86669 Alexander Ferry Suite 894,Justinborough,SC,57689,099.659.6812x370
4,Christopher,Stevens,M,1988-06-18,02510 Morgan Grove,New Wesley,IL,27745,372.203.6919x99940
5,Sarah,Hernandez,F,2002-12-31,0545 Andrew Square Suite 666,Port Caroline,SC,32894,+1-629-965-5817x702
6,Kathleen,Kerr,F,2002-01-18,9555 Danielle Views,Levineberg,DE,91849,352.923.4721
7,Jennifer,Flores,F,2003-07-03,5275 David Vista Apt. 787,South Lindseychester,OK,85429,764-507-6542x036
8,Brenda,Ward,F,1986-02-14,260 Bryan Crossing Suite 109,Mcknightland,KY,83326,001-191-428-4607x989
9,Dawn,Tucker,F,1998-11-27,3860 Edwards Coves Suite 164,Janiceburgh,DC,88121,099-802-4191


In [17]:
Student() & 'sex="F"'

student_id university-wide ID number,first_name,last_name,sex,date_of_birth,home_address mailing street address,home_city mailing address,home_state US state acronym: e.g. OH,home_zip zipcode e.g. 93979-4979,home_phone e.g. 414.657.6883x0881
1,Jamie,Gomez,F,1991-12-31,340 Manuel Lake Apt. 728,North William,MI,94138,522.492.8344
5,Sarah,Hernandez,F,2002-12-31,0545 Andrew Square Suite 666,Port Caroline,SC,32894,+1-629-965-5817x702
6,Kathleen,Kerr,F,2002-01-18,9555 Danielle Views,Levineberg,DE,91849,352.923.4721
7,Jennifer,Flores,F,2003-07-03,5275 David Vista Apt. 787,South Lindseychester,OK,85429,764-507-6542x036
8,Brenda,Ward,F,1986-02-14,260 Bryan Crossing Suite 109,Mcknightland,KY,83326,001-191-428-4607x989
9,Dawn,Tucker,F,1998-11-27,3860 Edwards Coves Suite 164,Janiceburgh,DC,88121,099-802-4191
10,Kristen,Lee,F,1996-04-23,83475 Long Gardens Apt. 530,Coffeymouth,KS,14920,(497)403-5362x2571
12,Sarah,George,F,1984-10-12,5578 Timothy Highway,Heatherchester,CA,42621,(141)751-5147x7784
14,Carol,Lopez,F,1999-09-01,07190 Rodriguez Track Apt. 046,South Sallyburgh,GA,53493,292-555-7519x95284
17,Melissa,Morgan,F,1994-06-28,55261 Christian Crossroad,North Steve,WA,33935,001-064-837-6776x963


In [18]:
female_texans = Student & {'sex': 'F', 'home_state': 'TX'}

In [19]:
female_texans

student_id university-wide ID number,first_name,last_name,sex,date_of_birth,home_address mailing street address,home_city mailing address,home_state US state acronym: e.g. OH,home_zip zipcode e.g. 93979-4979,home_phone e.g. 414.657.6883x0881
67,Tammy,Bell,F,1987-09-22,33212 Paul Forest,Brandonbury,TX,62769,001.788.4475
70,Amy,Ortiz,F,1998-09-21,7935 Lee Ramp,Johnborough,TX,82505,031.119.6332x9964
523,Tina,Walker,F,2003-02-28,1370 Gregory Trafficway Suite 530,North Brandon,TX,70175,999-938-3346x76896
544,Jillian,Holmes,F,1990-02-27,86858 Anderson Pike,Lake Christina,TX,29439,(481)498-3884


In [20]:
female_texans.fetch()

array([( 67, 'Tammy', 'Bell', 'F', datetime.date(1987, 9, 22), '33212 Paul Forest', 'Brandonbury', 'TX', '62769', '001.788.4475'),
       ( 70, 'Amy', 'Ortiz', 'F', datetime.date(1998, 9, 21), '7935 Lee Ramp', 'Johnborough', 'TX', '82505', '031.119.6332x9964'),
       (523, 'Tina', 'Walker', 'F', datetime.date(2003, 2, 28), '1370 Gregory Trafficway Suite 530', 'North Brandon', 'TX', '70175', '999-938-3346x76896'),
       (544, 'Jillian', 'Holmes', 'F', datetime.date(1990, 2, 27), '86858 Anderson Pike', 'Lake Christina', 'TX', '29439', '(481)498-3884')],
      dtype=[('student_id', '<i8'), ('first_name', 'O'), ('last_name', 'O'), ('sex', 'O'), ('date_of_birth', 'O'), ('home_address', 'O'), ('home_city', 'O'), ('home_state', 'O'), ('home_zip', 'O'), ('home_phone', 'O')])

In [21]:
female_texans.fetch(as_dict=True)

[{'student_id': 67,
  'first_name': 'Tammy',
  'last_name': 'Bell',
  'sex': 'F',
  'date_of_birth': datetime.date(1987, 9, 22),
  'home_address': '33212 Paul Forest',
  'home_city': 'Brandonbury',
  'home_state': 'TX',
  'home_zip': '62769',
  'home_phone': '001.788.4475'},
 {'student_id': 70,
  'first_name': 'Amy',
  'last_name': 'Ortiz',
  'sex': 'F',
  'date_of_birth': datetime.date(1998, 9, 21),
  'home_address': '7935 Lee Ramp',
  'home_city': 'Johnborough',
  'home_state': 'TX',
  'home_zip': '82505',
  'home_phone': '031.119.6332x9964'},
 {'student_id': 523,
  'first_name': 'Tina',
  'last_name': 'Walker',
  'sex': 'F',
  'date_of_birth': datetime.date(2003, 2, 28),
  'home_address': '1370 Gregory Trafficway Suite 530',
  'home_city': 'North Brandon',
  'home_state': 'TX',
  'home_zip': '70175',
  'home_phone': '999-938-3346x76896'},
 {'student_id': 544,
  'first_name': 'Jillian',
  'last_name': 'Holmes',
  'sex': 'F',
  'date_of_birth': datetime.date(1990, 2, 27),
  'home_addr

In [22]:
female_texans.fetch(format="frame")

,first_name,last_name,sex,date_of_birth,home_address,home_city,home_state,home_zip,home_phone
student_id,,,,,,,,,
67,Tammy,Bell,F,1987-09-22,33212 Paul Forest,Brandonbury,TX,62769,001.788.4475
70,Amy,Ortiz,F,1998-09-21,7935 Lee Ramp,Johnborough,TX,82505,031.119.6332x9964
523,Tina,Walker,F,2003-02-28,1370 Gregory Trafficway Suite 530,North Brandon,TX,70175,999-938-3346x76896
544,Jillian,Holmes,F,1990-02-27,86858 Anderson Pike,Lake Christina,TX,29439,(481)498-3884


In [23]:
len(female_texans)

4

In [24]:
utahns = Student & {'home_state': 'UT'}

In [25]:
utah_genz = utahns & 'date_of_birth > "1996-01-01"'

In [26]:
utah_genz

student_id university-wide ID number,first_name,last_name,sex,date_of_birth,home_address mailing street address,home_city mailing address,home_state US state acronym: e.g. OH,home_zip zipcode e.g. 93979-4979,home_phone e.g. 414.657.6883x0881
82,Mrs.,Elizabeth,F,1999-09-25,9701 Hernandez Points Apt. 045,East Reginald,UT,02278,001-265-808-3043
235,Amanda,Bates,F,2003-10-05,759 Michelle Parkway,South Tonyaville,UT,19461,001-284-444-2639x670
489,Shaun,Delacruz,M,1997-08-23,00848 Brandon Streets Apt. 387,Port Matthewton,UT,47087,(707)798-2524x905


In [27]:
utah_genz = utah_genz.proj('last_name', 'home_city','home_state','sex')
utah_genz

student_id university-wide ID number,last_name,sex,home_city mailing address,home_state US state acronym: e.g. OH
82,Elizabeth,F,East Reginald,UT
235,Bates,F,South Tonyaville,UT
489,Delacruz,M,Port Matthewton,UT


In [28]:
for r in utah_genz:
    print(r)

{'student_id': 82, 'last_name': 'Elizabeth', 'sex': 'F', 'home_city': 'East Reginald', 'home_state': 'UT'}
{'student_id': 235, 'last_name': 'Bates', 'sex': 'F', 'home_city': 'South Tonyaville', 'home_state': 'UT'}
{'student_id': 489, 'last_name': 'Delacruz', 'sex': 'M', 'home_city': 'Port Matthewton', 'home_state': 'UT'}
